In [33]:
# Install library yang diperlukan
import sqlite3
import pandas as pd
from google.colab import files

In [26]:
# Step 1: Unggah file Excel
print("Unggah file Excel (customers.xlsx, orders.xlsx, products.xlsx, product_categories.xlsx)")
uploaded = files.upload()

Unggah file Excel (customers.xlsx, orders.xlsx, products.xlsx, product_categories.xlsx)


Saving Products.xlsx to Products.xlsx
Saving ProductCategory.xlsx to ProductCategory.xlsx
Saving Customers.xlsx to Customers.xlsx
Saving Orders.xlsx to Orders.xlsx


In [91]:
# Step 2: Baca file Excel ke DataFrame
customers = pd.read_excel('Customers.xlsx')
orders = pd.read_excel('Orders.xlsx')
products = pd.read_excel('Products.xlsx')
product_categories = pd.read_excel('ProductCategory.xlsx')

In [103]:
# Check if 'CategoryID' column exists in 'products' DataFrame
if 'CategoryID' not in products.columns:
    # If not present, assuming it's named 'CatID' and renaming it
    products = products.rename(columns={'CatID': 'CategoryID'})
    # If the column is named something else, change 'CatID' to the actual name
    # Or, if the column is missing, you need to add it to the DataFrame

In [113]:
# Step 3: Buat koneksi ke SQLite
conn = sqlite3.connect('SalesData.db')
cursor = conn.cursor()

In [114]:
# Drop the existing 'products' table if it exists
cursor.execute("DROP TABLE IF EXISTS products")
conn.commit() # Commit the change

In [115]:
# Query to retrieve column names from the 'orders' table
cursor = conn.cursor()
cursor.execute("PRAGMA table_info(orders)")
columns = [column[1] for column in cursor.fetchall()]

In [116]:
# Print the column names
print(columns)

['OrderID', 'Date', 'CustomerID', 'ProdNumber', 'Quantity']


In [117]:
# Step 4: Simpan data ke SQLite
customers.to_sql('customers', conn, if_exists='replace', index=False)
orders.to_sql('orders', conn, if_exists='replace', index=False)
products.to_sql('products', conn, if_exists='replace', index=False)
product_categories.to_sql('product_categories', conn, if_exists='replace', index=False)

print("Data berhasil disimpan ke database SQLite.")

Data berhasil disimpan ke database SQLite.


In [120]:
# Query untuk membuat master table
query = """
    SELECT
        c.CustomerEmail AS cust_email,
        c.CustomerCity AS cust_city,
        o.Date AS order_date,
        o.Quantity AS order_qty,
        p.ProdName AS product_name,
        p.Price AS product_price,
        pc.CategoryName AS category_name,
        (o.Quantity * p.Price) AS total_sales
    FROM orders o
    JOIN customers c ON o.CustomerID = c.CustomerID
    JOIN products p ON o.ProdNumber = p.ProdNumber
    JOIN product_categories pc ON p.Category = pc.CategoryID
    ORDER BY o.Date
"""

In [121]:
# Eksekusi query dan simpan hasil ke DataFrame
master_table = pd.read_sql(query, conn)

In [122]:
# Tampilkan hasil master data
print("Tabel Master Data:")
print(master_table)


Tabel Master Data:
                                             cust_email         cust_city  \
0     fvaslerqt@comsenz.com#mailto:fvaslerqt@comsenz...           Jackson   
1     tmckernot@tinyurl.com#mailto:tmckernot@tinyurl...              Katy   
2     gstiggersdd@eventbrite.com#mailto:gstiggersdd@...  Saint Petersburg   
3                     edew@nba.com#mailto:edew@nba.com#          Honolulu   
4           llespercx@com.com#mailto:llespercx@com.com#        Des Moines   
...                                                 ...               ...   
3334  rcossumba@devhub.com#mailto:rcossumba@devhub.com#       Saint Louis   
3335  ngysehg@pagesperso-orange.fr#mailto:ngysehg@pa...          Richmond   
3336  kcoytenm@bandcamp.com#mailto:kcoytenm@bandcamp...      Philadelphia   
3337  nmotherwello3@istockphoto.com#mailto:nmotherwe...           Atlanta   
3338  ccollopoi@delicious.com#mailto:ccollopoi@delic...        Pittsburgh   

               order_date  order_qty              produc

In [123]:
# Simpan master table ke database SQLite
master_table.to_sql('master_table', conn, if_exists='replace', index=False)

3339

In [124]:
# Ekspor database sebagai file ZIP
!zip -r SalesData.zip SalesData.db
files.download("SalesData.zip")

  adding: SalesData.db (deflated 68%)


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>